In [2]:
# Algoritmo de Búsqueda Lineal de Newton con Modificación a la Hessiana.

    # Algoritmos 3.1 y 3.2 de Nocedal

import numpy as np
print(np.__version__)

1.19.2


In [85]:
def gradiente(func,xk):
    # Código que calcula el gradiente de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk array y lo convertimos a matriz de nx1
    xk = np.transpose(np.matrix(xk))
    cambio = .000001
    n = len(xk)
    grad = np.zeros((n,1))
    
    for i in range(1,n+1):
        aux = xk.copy()
        aux[i-1,0] = aux[i-1] + cambio
        grad[i-1,0] = round((func(aux) - func(xk))/cambio,2)
    
    return grad

In [84]:
xk = np.array([2.0,1.0])
gradiente(Rosenbrock,xk)

array([[2402.],
       [-600.]])

In [88]:
def hessiana(func,xk):
    # Código que calcula la hessiana de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk matrix de nx1
    cambio = .000001
    n = len(xk)
    hess = np.zeros((n,n))
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i==j:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux2[i-1,0] = aux2[i-1,0] - cambio
                hess[i-1,j-1] = round((func(aux1)-2*func(xk)+func(aux2))/cambio**2,2)
            else:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux3 = xk.copy()
                
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux1[j-1,0] = aux1[j-1,0] + cambio
                
                aux2[j-1,0] = aux2[j-1,0] + cambio
                
                aux3[i-1,0] = aux3[i-1,0] + cambio
                
                hess[i-1,j-1] = round((func(aux1)-func(aux2)-func(aux3)+func(xk))/cambio**2,2)
                
    return hess

In [134]:
xk = np.matrix([[2.0],[1.0]])
hessiana(Rosenbrock,xk)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [86]:
# Función de Prueba de los Algoritmos (Rosenbrock)
def Rosenbrock(x):
    # Aquí suponemos que x ya es una matriz (vector) de nx1
    return (1-x[0,0])**2 + 100*(x[1,0]-x[0,0]**2)**2

In [127]:
xk = np.matrix([[1.0],[3.0]])
Rosenbrock(xk)

400.0

In [31]:
# Algoritmo 3.1: Backtracking Line Search

def BLS(func,xk,pk,alpha,rho,c):
    a = alpha
    while func(xk+a*pk) > (func(xk) + c*a*gradiente(func,xk)*np.transpose(np.matrix(pk))):
        a = rho*a
    return a

In [36]:
xk = np.array([1.0,1.0])
pk = np.array([1.0,0.0])
rho = .6
c = .5
alpha = 1
BLS(Rosenbrock,xk,pk,alpha,rho,c)

1.0638735892371625e-16

In [55]:
# Algoritmo 3.2: Line Search Newton with Modification

def LSNM(func,x0):
    xk = np.transpose(np.matrix(x0))
    for k in range(1,20):
        # Suponiendo que la hessiana sí es positiva definida (simétrica):
        Bk = hessiana(func,xk)
        pk = np.linalg.solve(Bk,-np.transpose(gradiente(func,xk)))
        xk = xk + BLS(func,xk,pk,1,.6,.5)*pk
        
    return xk

In [56]:
x0 = np.array([2.0,1.0])
LSNM(Rosenbrock,x0)

TypeError: type matrix doesn't define __round__ method